# Import

In [ ]:
import os
import sys

import numpy as np

from PIL import Image

import matplotlib.pyplot as plt

import pickle

sys.path.append('src')
from dataset import create_file_list

# Dataset

### PASCAL-Part

Read all pkl files of a specific category

In [ ]:
aeroplane_annotations_path = '/content/drive/MyDrive/IDL_group_project/Google_Colab/data/PASCAL-Part-semantic-pkl/aeroplane'
bicycle_annotations_path = '/content/drive/MyDrive/IDL_group_project/Google_Colab/data/PASCAL-Part-semantic-pkl/bicycle'
boat_annotations_path = '/content/drive/MyDrive/IDL_group_project/Google_Colab/data/PASCAL-Part-semantic-pkl/boat'
bus_annotations_path = '/content/drive/MyDrive/IDL_group_project/Google_Colab/data/PASCAL-Part-semantic-pkl/bus'
car_annotations_path = '/content/drive/MyDrive/IDL_group_project/Google_Colab/data/PASCAL-Part-semantic-pkl/car'
motorbike_annotations_path = '/content/drive/MyDrive/IDL_group_project/Google_Colab/data/PASCAL-Part-semantic-pkl/motorbike'
train_annotations_path = '/content/drive/MyDrive/IDL_group_project/Google_Colab/data/PASCAL-Part-semantic-pkl/train'
image_folder_dir = '/content/drive/MyDrive/IDL_group_project/Data/VOCdevkit/VOC2010/JPEGImages/'
target_labels = ['aeroplane', 'bicycle', 'boat', 'bus', 'car', 'motorbike', 'train']

In [ ]:
PASCAL_Part_semantic_pkl_dir = 'data/PASCAL-Part-semantic-pkl/' + category # Config the category in the Config section
semantic_file_name_list, semantic_file_path_list = create_file_list(PASCAL_Part_semantic_pkl_dir)

print('Total ' + category + ' images: ' + str(len(image_anno_file_name_list)))
print('First 5 samples:')
print(semantic_file_name_list[:5])
print(semantic_file_path_list[:5])

In [ ]:
#Concate all different classes semantic information together from seperate directories
def concate_directories(paths):
  semantic_bundles_paths_extended = []
  total_files = 0 

  for path in paths:
    class_bundle_paths , num_files = path_file_list_creator(path)
    semantic_bundles_paths_extended.extend(class_bundle_paths)
    total_files += num_files
  print("Total Files grabbed ", total_files)
  return(semantic_bundles_paths_extended)
  
class SemanticObjectsLoader(DataLoader):
  def __init__(self, paths, image_folder_path, target_lables):
    self.paths = concate_directories(paths)
    self.image_folder_dir = image_folder_path
    self.target_labels = target_labels

  def __iter__(self):
    for i, path in enumerate(self.paths):
      with open(path, 'rb') as f:
        data = pickle.load(f)
      image_name = data['image_name']
      #We must append the full path name to image_name
      image_name = self.image_folder_dir + image_name + '.jpg'
      num_objects = data["num_objects"]
      #we must iterate through every different object (for example there could be two airplanes in one photo)
      for object_index in range(num_objects):
        #Crawling through each object index
        label = data['objects_list'][object_index]['obj_class']
        if label not in self.target_labels:
          break
        #We must iterate through every semantic item now
        for semantic_index in range(len(data['objects_list'][object_index]['parts_list'])):
          semantic_mask = data['objects_list'][object_index]['parts_list'][semantic_index]['part_mask']
          semantic_type = data['objects_list'][object_index]['parts_list'][semantic_index]['part_name']
          #Fill in a new entry into the dictionary
          meta_dict = {"image_name": image_name, "label" : label, "semantic_mask" : semantic_mask, "semantic_type" :semantic_type}
          yield meta_dict

    yield full_semantic_list_getter(self.paths)

<!-- The output is a dictionary, access data as the following example -->